# Transformation of Santander dataset into Articles, Customers, Transaction (ACT) format

In [8]:
%load_ext kedro.extras.extensions.ipython

The kedro.extras.extensions.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.extras.extensions.ipython


In [9]:
%reload_kedro

2022-08-24 12:25:52,987 - kedro.framework.hooks.manager - INFO - Registered hooks from 1 installed plugin(s): kedro-mlflow-0.8.1
2022-08-24 12:25:53,127 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2022-08-24 12:25:53,191 - root - INFO - ** Kedro project GetInData ML Framework
2022-08-24 12:25:53,193 - root - INFO - Defined global variable `context`, `session`, `catalog` and `pipelines`
2022-08-24 12:25:53,216 - root - INFO - Registered line magic `run_viz`
2022-08-24 12:25:53,218 - root - INFO - Registered line magic `reload_kedro_mlflow`


/home/michal/projects/gid-ml-framework/.venv/lib/python3.8/site-packages/kedro/framework/context/context.py:488: UserWarning: Credentials not found in your Kedro project config.
No files found in ['/home/michal/projects/gid-ml-framework/conf/base', '/home/michal/projects/gid-ml-framework/conf/local'] matching the glob pattern(s): ['credentials*', 'credentials*/**', '**/credentials*']
  warn(f"Credentials not found in your Kedro project config.\n{str(exc)}")


In [50]:
from typing import Iterator, Tuple
import re

import pandas as pd
import numpy as np
from kedro.extras.datasets.pandas import CSVDataSet
from kedro.io.core import get_filepath_str

import gid_ml_framework.pipelines.santander_preprocessing.nodes
from gid_ml_framework.extras.datasets.chunks_dataset import (
 _load,
 _concat_chunks,
)

pd.options.mode.chained_assignment = None
pd.set_option('expand_frame_repr', True)
pd.set_option("display.max_rows", 999)
pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)

## Final dataframes check

In [73]:
santander_articles_path = "santander_to_act_main.santander_articles"
santander_customers_path = "santander_to_act_main.santander_customers"
santander_trainsactions_train_path = "santander_to_act_main.santander_transactions_train"
santander_trainsactions_val_path = "santander_to_act_main.santander_transactions_val"

In [74]:
santander_customers = _concat_chunks(context
                                     .catalog.load(santander_customers_path))
santander_articles = _concat_chunks(context
                                    .catalog.load(santander_articles_path))
santander_transactions_train = _concat_chunks(context
                                              .catalog
                                              .load(santander_trainsactions_train_path))
santander_transactions_val = _concat_chunks(context
                                              .catalog
                                              .load(santander_trainsactions_val_path))

2022-08-25 12:26:52,121 - kedro.io.data_catalog - INFO - Loading data from `santander_to_act_main.santander_customers` (CSVDataSet)...


/home/michal/projects/gid-ml-framework/.venv/lib/python3.8/site-packages/kedro/framework/context/context.py:488: UserWarning: Credentials not found in your Kedro project config.
No files found in ['/home/michal/projects/gid-ml-framework/conf/base', '/home/michal/projects/gid-ml-framework/conf/local'] matching the glob pattern(s): ['credentials*', 'credentials*/**', '**/credentials*']
  warn(f"Credentials not found in your Kedro project config.\n{str(exc)}")


2022-08-25 12:26:54,342 - kedro.io.data_catalog - INFO - Loading data from `santander_to_act_main.santander_articles` (CSVDataSet)...
2022-08-25 12:26:54,377 - kedro.io.data_catalog - INFO - Loading data from `santander_to_act_main.santander_transactions_train` (CSVDataSet)...
2022-08-25 12:26:54,441 - kedro.io.data_catalog - INFO - Loading data from `santander_to_act_main.santander_transactions_val` (CSVDataSet)...


/home/michal/projects/gid-ml-framework/.venv/lib/python3.8/site-packages/kedro/framework/context/context.py:488: UserWarning: Credentials not found in your Kedro project config.
No files found in ['/home/michal/projects/gid-ml-framework/conf/base', '/home/michal/projects/gid-ml-framework/conf/local'] matching the glob pattern(s): ['credentials*', 'credentials*/**', '**/credentials*']
  warn(f"Credentials not found in your Kedro project config.\n{str(exc)}")
/home/michal/projects/gid-ml-framework/.venv/lib/python3.8/site-packages/kedro/framework/context/context.py:488: UserWarning: Credentials not found in your Kedro project config.
No files found in ['/home/michal/projects/gid-ml-framework/conf/base', '/home/michal/projects/gid-ml-framework/conf/local'] matching the glob pattern(s): ['credentials*', 'credentials*/**', '**/credentials*']
  warn(f"Credentials not found in your Kedro project config.\n{str(exc)}")
/home/michal/projects/gid-ml-framework/.venv/lib/python3.8/site-packages/ked

In [13]:
santander_articles

,article_id
0,ind_ahor_fin_ult1
1,ind_aval_fin_ult1
2,ind_cco_fin_ult1
3,ind_cder_fin_ult1
4,ind_cno_fin_ult1
5,ind_ctju_fin_ult1
6,ind_ctma_fin_ult1
7,ind_ctop_fin_ult1
8,ind_ctpp_fin_ult1
9,ind_deco_fin_ult1


In [14]:
santander_customers

,customer_id,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,nomprov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,904057,N,ES,H,46,2011-02-15,0,62,1,UNKNOWN,1,A,S,N,UNKNOWN,KFC,N,MADRID,1,216306.203125,02 - PARTICULARES,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,903983,N,ES,H,63,2011-02-14,0,62,1,UNKNOWN,1,A,S,N,UNKNOWN,KFC,N,MADRID,1,348805.250000,02 - PARTICULARES,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,904048,N,ES,H,37,2011-02-15,0,62,1,UNKNOWN,1,I,S,N,UNKNOWN,KFC,N,MADRID,0,899925.312500,02 - PARTICULARES,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,904022,N,ES,H,39,2011-02-15,0,62,1,UNKNOWN,1,A,S,S,UNKNOWN,KAT,N,CADIZ,1,106174.742188,02 - PARTICULARES,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,902560,N,ES,V,29,2011-01-31,0,62,1,UNKNOWN,1,I,S,N,UNKNOWN,KFC,N,"PALMAS, LAS",0,130563.476562,02 - PARTICULARES,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92863,253075,N,ES,H,34,2005-10-26,0,54,1,UNKNOWN,1,A,S,N,UNKNOWN,KAT,N,ZARAGOZA,1,89344.773438,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,1
92864,253051,N,ES,V,82,2001-06-12,0,178,1,UNKNOWN,1,I,S,N,UNKNOWN,KFA,N,MADRID,1,91373.882812,02 - PARTICULARES,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
92865,253062,N,ES,V,51,2001-06-12,0,178,1,UNKNOWN,1,A,S,S,UNKNOWN,KAT,N,MADRID,0,73089.601562,02 - PARTICULARES,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
92866,252741,N,ES,H,79,2001-06-11,0,178,1,UNKNOWN,1,I,S,N,UNKNOWN,KAC,N,NAVARRA,1,54599.878906,02 - PARTICULARES,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [75]:
santander_transactions_train

,customer_id,date,article_id
0,906299,2015-04-28,ind_aval_fin_ult1
1,1278838,2015-02-28,ind_cco_fin_ult1
2,914458,2015-02-28,ind_cco_fin_ult1
3,302272,2015-02-28,ind_cco_fin_ult1
4,498142,2015-02-28,ind_cco_fin_ult1
...,...,...,...
52883,873770,2016-04-28,ind_recibo_ult1
52884,1025205,2016-04-28,ind_recibo_ult1
52885,143744,2016-04-28,ind_recibo_ult1
52886,874731,2016-04-28,ind_recibo_ult1


## Transactions transformations optimization

In [15]:
train_path = "santander_train"
val_path = "santander_val"
test_path = "santander_test"

In [16]:
train = _concat_chunks(context.catalog.load(train_path))
val = _concat_chunks(context.catalog.load(val_path))
test = _concat_chunks(context.catalog.load(test_path))

2022-08-24 14:51:26,717 - kedro.io.data_catalog - INFO - Loading data from `santander_train` (CSVDataSet)...


/home/michal/projects/gid-ml-framework/.venv/lib/python3.8/site-packages/kedro/framework/context/context.py:488: UserWarning: Credentials not found in your Kedro project config.
No files found in ['/home/michal/projects/gid-ml-framework/conf/base', '/home/michal/projects/gid-ml-framework/conf/local'] matching the glob pattern(s): ['credentials*', 'credentials*/**', '**/credentials*']
  warn(f"Credentials not found in your Kedro project config.\n{str(exc)}")


2022-08-24 14:51:27,054 - kedro.io.data_catalog - INFO - Loading data from `santander_val` (CSVDataSet)...
2022-08-24 14:51:27,103 - kedro.io.data_catalog - INFO - Loading data from `santander_test` (CSVDataSet)...


/home/michal/projects/gid-ml-framework/.venv/lib/python3.8/site-packages/kedro/framework/context/context.py:488: UserWarning: Credentials not found in your Kedro project config.
No files found in ['/home/michal/projects/gid-ml-framework/conf/base', '/home/michal/projects/gid-ml-framework/conf/local'] matching the glob pattern(s): ['credentials*', 'credentials*/**', '**/credentials*']
  warn(f"Credentials not found in your Kedro project config.\n{str(exc)}")
/home/michal/projects/gid-ml-framework/.venv/lib/python3.8/site-packages/kedro/framework/context/context.py:488: UserWarning: Credentials not found in your Kedro project config.
No files found in ['/home/michal/projects/gid-ml-framework/conf/base', '/home/michal/projects/gid-ml-framework/conf/local'] matching the glob pattern(s): ['credentials*', 'credentials*/**', '**/credentials*']
  warn(f"Credentials not found in your Kedro project config.\n{str(exc)}")


In [23]:
def _status_change(x: pd.Series) -> str:
    """Based on difference of the following rows create label which indicates
    if given product was added, dropped or maintained in comparison with last 
    month for given customer.

    Args:
        x (pd.DataFrame): imputed santander train dataframe

    Returns:
        str: target label - added/dropped/maintained
    """
    diffs = x.diff().fillna(0)
    # First occurrence is considered as "Maintained"
    label = ["Added" if i == 1 \
         else "Dropped" if i == -1 \
         else "Maintained" for i in diffs]
    return label


def _target_processing_santander(train_df: Iterator[pd.DataFrame],
                                val_df: Iterator[pd.DataFrame]) -> Tuple:
    """Preprocess target columns to focus on products that will be bought
    in the next month

    Args:
        input_train_df (Iterator[pd.DataFrame]): imputed santander train
        dataframe
        input_val_df (Iterator[pd.DataFrame]): imputed santander validation 
        dataframe

    Returns:
        Tuple: processed train and validation dataframes
    """
    train_len = len(train_df)
    df = pd.concat([train_df, val_df])
    feature_cols = df.iloc[:1,].filter(regex="ind_+.*ult.*").columns.values
    df = df.sort_values(['ncodpers', 'fecha_dato']).reset_index(drop=True)
    # Apply status change labeling
    df.loc[:, feature_cols] = (df.loc[:, [i for i in feature_cols]
                               + ["ncodpers"]].groupby("ncodpers")
                               .transform(_status_change))
                               
    df = df.sort_values(['fecha_dato']).reset_index(drop=True)

    train_df = df.iloc[:train_len, :]
    val_df = df.iloc[train_len:, :]
    return (train_df, val_df)

In [26]:
transactions_train_1, transactions_val_1 = _target_processing_santander(train, val)

In [31]:
transactions_train_1

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,nomprov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-01-28,15930,N,ES,H,51,1996-01-24,0,246,1,UNKNOWN,1,A,S,N,N,KAT,N,MADRID,1,246128.343750,01 - TOP,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained
1,2015-01-28,968140,N,ES,H,24,2011-10-24,0,45,1,UNKNOWN,1,A,S,N,UNKNOWN,KHE,N,"CORUÑA, A",1,115836.812500,03 - UNIVERSITARIO,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained
2,2015-01-28,967271,N,ES,H,23,2011-10-21,0,45,1,UNKNOWN,1,I,S,N,UNKNOWN,KHE,N,TOLEDO,0,94542.687500,03 - UNIVERSITARIO,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained
3,2015-01-28,967227,N,ES,V,24,2011-10-21,0,45,1,UNKNOWN,1,I,S,N,UNKNOWN,KHE,N,MADRID,0,502885.593750,03 - UNIVERSITARIO,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained
4,2015-01-28,965393,N,ES,H,26,2011-10-19,0,45,1,UNKNOWN,1,I,S,N,UNKNOWN,KHE,N,NAVARRA,0,107442.812500,03 - UNIVERSITARIO,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12786,2016-04-28,51984,N,ES,H,52,1996-12-16,0,232,1,UNKNOWN,1,I,S,N,UNKNOWN,KAT,N,MADRID,0,133079.812500,02 - PARTICULARES,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained
12787,2016-04-28,538580,N,ES,V,29,2005-06-24,0,129,99,2016-04-05,1,A,S,N,UNKNOWN,KAT,N,SEVILLA,0,160014.031250,03 - UNIVERSITARIO,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained
12788,2016-04-28,1432694,N,ES,V,20,2015-08-10,0,8,1,UNKNOWN,1,I,S,N,UNKNOWN,KHQ,N,BARCELONA,0,394540.031250,03 - UNIVERSITARIO,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained
12789,2016-04-28,1232550,N,ES,V,22,2013-12-18,0,28,1,UNKNOWN,1,A,S,N,UNKNOWN,KH

In [35]:
transactions_train_1.loc[transactions_train_1.loc[:, 'ind_cco_fin_ult1'] == 'Added'].sort_values('ncodpers').head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,nomprov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
8466,2015-12-28,348324,N,ES,H,42,2002-04-11,0,68,1,UNKNOWN,1,A,S,N,UNKNOWN,KAT,N,MADRID,1,115269.421875,02 - PARTICULARES,Maintained,Maintained,Added,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained
2922,2015-05-28,360274,N,ES,V,20,2002-05-21,0,158,1,UNKNOWN,1,A,S,N,UNKNOWN,KAT,N,MADRID,1,73002.148438,02 - PARTICULARES,Maintained,Maintained,Added,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained
5472,2015-09-28,387803,N,ES,V,25,2002-10-28,1,5,1,UNKNOWN,1,A,S,N,UNKNOWN,KHN,N,MADRID,1,198122.156250,02 - PARTICULARES,Maintained,Maintained,Added,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained
8202,2015-12-28,410712,N,ES,V,40,2003-03-11,0,153,1,UNKNOWN,1,A,S,N,UNKNOWN,KFC,N,MADRID,1,39968.699219,02 - PARTICULARES,Maintained,Maintained,Added,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained
5094,2015-08-28,446190,N,ES,H,41,2003-11-24,0,141,1,UNKNOWN,1,I,S,N,UNKNOWN,KFC,N,MADRID,1,283676.343750,02 - PARTICULARES,Maintained,Maintained,Added,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Added,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained


In [37]:
train.loc[train.ncodpers == 360274, :]

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,nomprov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
128,2015-01-28,360274,N,ES,V,20,2002-05-21,0,158,1,UNKNOWN,1,A,S,N,UNKNOWN,KAT,N,MADRID,1,73002.148438,02 - PARTICULARES,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1119,2015-02-28,360274,N,ES,V,20,2002-05-21,0,158,1,UNKNOWN,1,A,S,N,UNKNOWN,KAT,N,MADRID,1,73002.148438,02 - PARTICULARES,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1741,2015-03-28,360274,N,ES,V,20,2002-05-21,0,158,1,UNKNOWN,1,A,S,N,UNKNOWN,KAT,N,MADRID,1,73002.148438,02 - PARTICULARES,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2370,2015-04-28,360274,N,ES,V,20,2002-05-21,0,158,1,UNKNOWN,1,A,S,N,UNKNOWN,KAT,N,MADRID,1,73002.148438,02 - PARTICULARES,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2632,2015-05-28,360274,N,ES,V,20,2002-05-21,0,158,1,UNKNOWN,1,A,S,N,UNKNOWN,KAT,N,MADRID,1,73002.148438,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3262,2015-06-28,360274,N,ES,V,20,2002-05-21,0,158,1,UNKNOWN,1,A,S,N,UNKNOWN,KAT,N,MADRID,1,73002.148438,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3907,2015-07-28,360274,N,ES,V,20,2002-05-21,0,158,1,UNKNOWN,1,A,S,N,UNKNOWN,KAT,N,MADRID,1,73002.148438,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4821,2015-08-28,360274,N,ES,V,20,2002-05-21,0,159,1,UNKNOWN,1,A,S,N,UNKNOWN,KAT,N,MADRID,1,73002.148438,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
6159,2015-09-28,360274,N,ES,V,20,2002-05-21,0,160,1,UNKNOWN,1,A,S,N,UNKNOWN,KAT,N,MADRID,1,73002.148438,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
6529,2015-10-28,360274,N,ES,V,20,2002-05-21,0,161,1,UNKNOWN,1,A,S,N,UNKNOWN,KAT,N,MADRID,1,73002.148438,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1


In [43]:
def _status_change_faster(x: pd.Series) -> str:
    """Based on difference of the following rows create label which indicates
    if given product was added, dropped or maintained in comparison with last 
    month for given customer.

    Args:
        x (pd.DataFrame): imputed santander train dataframe

    Returns:
        str: target label - added/dropped/maintained
    """
    # First occurrence is considered as "Maintained"
    label = ["Added" if i == 1 \
         else "Dropped" if i == -1 \
         else "Maintained" for i in x]
    return label


def _target_processing_santander_faster(train_df: Iterator[pd.DataFrame],
                                val_df: Iterator[pd.DataFrame]) -> Tuple:
    """Preprocess target columns to focus on products that will be bought
    in the next month

    Args:
        input_train_df (Iterator[pd.DataFrame]): imputed santander train
        dataframe
        input_val_df (Iterator[pd.DataFrame]): imputed santander validation 
        dataframe

    Returns:
        Tuple: processed train and validation dataframes
    """
    train_len = len(train_df)
    df = pd.concat([train_df, val_df])
    feature_cols = df.iloc[:1,].filter(regex="ind_+.*ult.*").columns.values
    df = df.sort_values(['ncodpers', 'fecha_dato']).reset_index(drop=True)
    # Apply status change labeling
    s = df.loc[:, [i for i in feature_cols] + ["ncodpers"]]
    df.loc[:, feature_cols] = (s.loc[:, [i for i in feature_cols]].diff()
                               .where(s.duplicated(["ncodpers"],
                               keep='first'))
                               ).fillna(0).transform(_status_change_faster)
    df = df.sort_values(['fecha_dato']).reset_index(drop=True)

    train_df = df.iloc[:train_len, :]
    val_df = df.iloc[train_len:, :]
    return (train_df, val_df)

In [44]:
transactions_train_2, transactions_val_2 = _target_processing_santander_faster(train, val)

In [45]:
transactions_train_2.loc[transactions_train_2.loc[:, 'ind_cco_fin_ult1'] == 'Added'].sort_values('ncodpers').head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,nomprov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
8466,2015-12-28,348324,N,ES,H,42,2002-04-11,0,68,1,UNKNOWN,1,A,S,N,UNKNOWN,KAT,N,MADRID,1,115269.421875,02 - PARTICULARES,Maintained,Maintained,Added,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained
2922,2015-05-28,360274,N,ES,V,20,2002-05-21,0,158,1,UNKNOWN,1,A,S,N,UNKNOWN,KAT,N,MADRID,1,73002.148438,02 - PARTICULARES,Maintained,Maintained,Added,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained
5472,2015-09-28,387803,N,ES,V,25,2002-10-28,1,5,1,UNKNOWN,1,A,S,N,UNKNOWN,KHN,N,MADRID,1,198122.156250,02 - PARTICULARES,Maintained,Maintained,Added,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained
8202,2015-12-28,410712,N,ES,V,40,2003-03-11,0,153,1,UNKNOWN,1,A,S,N,UNKNOWN,KFC,N,MADRID,1,39968.699219,02 - PARTICULARES,Maintained,Maintained,Added,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained
5094,2015-08-28,446190,N,ES,H,41,2003-11-24,0,141,1,UNKNOWN,1,I,S,N,UNKNOWN,KFC,N,MADRID,1,283676.343750,02 - PARTICULARES,Maintained,Maintained,Added,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Added,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained,Maintained


In [46]:
train.loc[train.ncodpers == 348324, :]

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,nomprov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
78,2015-01-28,348324,N,ES,H,41,2002-04-11,0,63,1,UNKNOWN,1,A,S,N,UNKNOWN,KAT,N,MADRID,1,115269.421875,02 - PARTICULARES,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1116,2015-02-28,348324,N,ES,H,41,2002-04-11,0,63,1,UNKNOWN,1,A,S,N,UNKNOWN,KAT,N,MADRID,1,115269.421875,02 - PARTICULARES,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1755,2015-03-28,348324,N,ES,H,41,2002-04-11,0,63,1,UNKNOWN,1,A,S,N,UNKNOWN,KAT,N,MADRID,1,115269.421875,02 - PARTICULARES,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2425,2015-04-28,348324,N,ES,H,41,2002-04-11,0,63,1,UNKNOWN,1,A,S,N,UNKNOWN,KAT,N,MADRID,1,115269.421875,02 - PARTICULARES,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2637,2015-05-28,348324,N,ES,H,41,2002-04-11,0,63,1,UNKNOWN,1,A,S,N,UNKNOWN,KAT,N,MADRID,1,115269.421875,02 - PARTICULARES,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3266,2015-06-28,348324,N,ES,H,41,2002-04-11,0,63,1,UNKNOWN,1,A,S,N,UNKNOWN,KAT,N,MADRID,1,115269.421875,02 - PARTICULARES,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3910,2015-07-28,348324,N,ES,H,41,2002-04-11,0,63,1,UNKNOWN,1,A,S,N,UNKNOWN,KAT,N,MADRID,1,115269.421875,02 - PARTICULARES,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4843,2015-08-28,348324,N,ES,H,41,2002-04-11,0,64,1,UNKNOWN,1,A,S,N,UNKNOWN,KAT,N,MADRID,1,115269.421875,02 - PARTICULARES,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
6300,2015-09-28,348324,N,ES,H,41,2002-04-11,0,65,1,UNKNOWN,1,A,S,N,UNKNOWN,KAT,N,MADRID,1,115269.421875,02 - PARTICULARES,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
6498,2015-10-28,348324,N,ES,H,41,2002-04-11,0,66,1,UNKNOWN,1,A,S,N,UNKNOWN,KAT,N,MADRID,1,115269.421875,02 - PARTICULARES,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [48]:
transactions_train_2.equals(transactions_train_1)
transactions_val_2.equals(transactions_val_1)

True